In [41]:
%pip install crewai 'crewai[tools]' groq tavily-python exa_py --quiet 

In [42]:
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew, LLM
from dotenv import load_dotenv

load_dotenv()

llm = LLM("groq/llama-3.1-8b-instant", temperature=0)

In [43]:
from crewai_tools import TXTSearchTool

with open("../health_data.txt", "r") as f:
    content = f.read()
    
txt_tool = TXTSearchTool(
    txt= content,
)

In [44]:
from exa_py import Exa
import os
from crewai_tools import tool


exa_api_key = os.getenv("EXA_API_KEY")

@tool("Exa Reddit search")
def search_reddit_posts(query: str, num_results: int = 5) -> str:
    """"
    Search Reddit posts using Exa's semantic search capabilities.
    
    Args:
        query (str): The search query.
        num_results (int): Number of results to retrieve.
    
    Returns:
        str: Formatted string containing the Reddit post details.
    """
    exa = Exa(exa_api_key)

    search_query = f"site:reddit.com {query}"

    response = exa.search_and_contents(
        search_query,
        type="neural",          
        use_autoprompt=True,   
        num_results=num_results,
        include_domains=["Reddit.com"],
        highlights=True
    )

    if not response.results:
        return "No results found."

    parsed_results = ''.join([
        f"<Title id={idx}>{result.title}</Title>\n"
        f"<URL id={idx}>{result.url}</URL>\n"
        f"<Highlight id={idx}>{' '.join(result.highlights)}</Highlight>\n\n"
        for idx, result in enumerate(response.results)
    ])

    return parsed_results

exa_tool = search_reddit_posts

In [45]:
health_agent = Agent(
    role="Smart Health Context Analyzer",
    goal="Determine if query is user-specific  (e.g., contains 'my condition,' 'for me') and if necessary provide user medical context",
    tools = [txt_tool],
    verbose=True,
    backstory=(
        "You analyze patient data"
        "to determine if any relevant information exists for the query."
        "However, only do so if the query is user-specific  (e.g., contains 'my condition,' 'for me')"
    ),
    llm=LLM(model="gpt-4o", temperature=0),
)

In [46]:
search_agent = Agent(
    role="Senior Reddit Researcher",
    goal="Use Exa Reddit Search for relevant information for the query.",
    tools = [exa_tool],
    verbose=True,
    backstory=(
        "You are skilled at finding reliable medical information from Reddit.com, ONLY search this domain"
    ),
    llm=LLM(model="gpt-4o", temperature=0)
)

In [47]:
synthesis_agent = Agent(
    role="Professional Medical Information Synthesizer",
    goal="Combine results from Reddit searches, health data, and other sources into a coherent response to the query",
    tools = [],
    verbose=True,
    backstory=(
        "You synthesize data from multiple sources to create clear and concise answers for the user."
    ),
    llm=LLM(model="gpt-4o", temperature=0)
)

In [48]:
health_context_task = Task(
    description=(
        "Check if the query {query} EXPLICITLY references the user's health (e.g., contains 'my condition,' 'for me')"
        "If NO, then return '' and ignore the rest of task. If YES, proceed to the following steps: "
        "Using the tool, Provide relevant medical context to help answer the query"
        "Ensure there's no hallucination and include citations for each piece of information"
    ),
    expected_output=(
        "A structured health context report."
    ),
    agent=health_agent,
    async_execution=True
)

In [49]:
search_task = Task(
    description=(
        "Search Reddit for information about: {query}\n"
        "Use the exa_tool to retrieve information"
        "Your task:"
        "1. Search to provide relevant information from Reddit.com\n"
        "3. Return your results"
    ),
    expected_output=(
        "A structured report from Reddit with clear citations."
    ),
    agent=search_agent,
    async_execution=True
)

In [50]:
synthesis_task = Task(
    description=(
        "Create a comprehensive answer for: {query}"
        "Using:"
        "Health Context (if available)"
        "Reddit Search Results"
        "List all sources at the end, no need to include references"
        "Include a brief medical disclaimer"
        "Keep the response clear, and well-structured."
    ),
    expected_output=(
        "A clear medical response with proper organization and citations with links."
    ),
    context=[health_context_task, search_task],
    agent=synthesis_agent
)

In [51]:
health_crew = Crew(
    agents=[health_agent,
            search_agent,
            synthesis_agent,
    ],
    tasks=[health_context_task,
           search_task,
           synthesis_task,
    ],
    verbose=True
)

In [52]:
input = {'query':'What is an astrocytoma?'}
result = health_crew.kickoff(inputs=input)
from IPython.display import Markdown
Markdown(str(result))

# Agent: Senior Reddit Researcher
## Task: Search Reddit for information about: What is an astrocytoma?
Use the exa_tool to retrieve informationYour task:1. Search to provide relevant information from Reddit.com
3. Return your results
# Agent: Smart Health Context Analyzer
## Task: Check if the query What is an astrocytoma? EXPLICITLY references the user's health (e.g., contains 'my condition,' 'for me')If NO, then return '' and ignore the rest of task. If YES, proceed to the following steps: Using the tool, Provide relevant medical context to help answer the queryEnsure there's no hallucination and include citations for each piece of information


# Agent: Smart Health Context Analyzer
## Final Answer: 





# Agent: Senior Reddit Researcher
## Thought: Thought: I need to search Reddit for information about astrocytoma to provide a structured report with clear citations.
## Using tool: Exa Reddit search
## Tool Input: 
"{\"query\": \"What is an astrocytoma?\", \"num_results\": 5}"
## 

Astrocytoma is a type of brain tumor that originates from astrocytes, which are star-shaped glial cells in the brain and spinal cord. These cells are part of the supportive tissue of the brain, and their primary function is to provide nutrients to the nervous tissue, maintain the blood-brain barrier, and repair the brain and spinal cord following traumatic injuries. Astrocytomas can occur in both adults and children, and they vary in their level of malignancy, ranging from low-grade (less aggressive) to high-grade (more aggressive) tumors.

**Types of Astrocytomas:**
1. **Pilocytic Astrocytoma (Grade I):** These are slow-growing tumors that are often considered benign and are more common in children and young adults. They are typically well-circumscribed and can often be treated successfully with surgery.
   
2. **Diffuse Astrocytoma (Grade II):** These tumors grow more slowly than higher-grade astrocytomas but can become more aggressive over time. They are infiltrative, meaning they spread into surrounding brain tissue, making complete surgical removal challenging.

3. **Anaplastic Astrocytoma (Grade III):** These are malignant tumors that grow more rapidly and are more aggressive than lower-grade astrocytomas. They often require a combination of surgery, radiation, and chemotherapy for treatment.

4. **Glioblastoma (Grade IV):** Also known as glioblastoma multiforme (GBM), this is the most aggressive form of astrocytoma. It is highly malignant, grows rapidly, and has a poor prognosis. Treatment typically involves surgery, radiation, and chemotherapy, but the tumor often recurs.

**Genetic Complexity and Treatment Challenges:**
Recent research, as highlighted in a Reddit post from the r/science subreddit, indicates that astrocytomas can exhibit genetic differences between the primary tumor and its metastases. This genetic variability can complicate treatment, as therapies targeting the primary tumor may not be effective against metastatic sites. This underscores the importance of personalized medicine and the need for comprehensive genetic profiling of tumors to guide treatment decisions.

**Impact on Behavior:**
Astrocytomas can also impact behavior and cognitive function, depending on their location in the brain. For instance, a historical case discussed on Reddit involved Charles Whitman, who had a tumor pressing on his brain, potentially influencing his behavior. This highlights the complex relationship between brain tumors and neurological symptoms.

**Medical Disclaimer:**
This information is provided for educational purposes only and is not intended as medical advice. For specific medical concerns, please consult a healthcare professional.

Sources:
- Reddit posts discussing astrocytomas and their genetic complexity.
- General medical knowledge on astrocytomas and their classification.